# Домашка

In [3]:
#!pip install requests
#!pip install apimoex

In [121]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import datetime
import numpy as np
import time
pd.options.display.float_format = '{:,.3f}'.format

# Выгрузка данных акций 

In [6]:
base_url = 'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json'

In [7]:
kapital = requests.get(base_url)

In [8]:
columns = kapital.json()['securities']['columns']
info = kapital.json()['securities']['data']

In [284]:
data_of_companies = []
for i in info:
    company = dict()
    for j in range(len(columns)):
        company[columns[j]] = i[j]
    data_of_companies.append(company)

In [286]:
data_of_companies[0]

{'SECID': 'ABIO',
 'BOARDID': 'TQBR',
 'SHORTNAME': 'iАРТГЕН ао',
 'PREVPRICE': 85.62,
 'LOTSIZE': 10,
 'FACEVALUE': 0.1,
 'STATUS': 'A',
 'BOARDNAME': 'Т+: Акции и ДР - безадрес.',
 'DECIMALS': 2,
 'SECNAME': 'ПАО "Артген"',
 'REMARKS': None,
 'MARKETCODE': 'FNDT',
 'INSTRID': 'EQIN',
 'SECTORID': None,
 'MINSTEP': 0.02,
 'PREVWAPRICE': 85.9,
 'FACEUNIT': 'SUR',
 'PREVDATE': '2024-08-01',
 'ISSUESIZE': 92645451,
 'ISIN': 'RU000A0JNAB6',
 'LATNAME': 'ARTGEN ao',
 'REGNUMBER': '1-01-08902-A',
 'PREVLEGALCLOSEPRICE': 85.76,
 'CURRENCYID': 'SUR',
 'SECTYPE': '1',
 'LISTLEVEL': 2,
 'SETTLEDATE': '2024-08-05'}

In [288]:
df_companies = pd.DataFrame({
    'SECID': [i['SECID'] for i in our_comp],
    'NAME': [i['SHORTNAME'] for i in our_comp], 
    'Market_Cap':[(i['PREVPRICE'] if i['PREVPRICE'] != None else 0) * (i['ISSUESIZE'] if i['ISSUESIZE'] != None else 0) \
                  for i in our_comp]
})

In [296]:
top_50 = df_companies.sort_values(by='Market_Cap', ascending=False).head(50) # топ 50

# Загружаем исторические данные стоимости топовых акций с 2015 год

In [300]:
top_id = top_50['SECID']

In [302]:
dates = pd.date_range(pd.to_datetime('2015-01-01'), datetime.today().strftime('%Y-%m-%d'), freq = '100D', normalize = True)

In [304]:
dates = dates.strftime('%Y-%m-%d')

In [306]:
dates

Index(['2015-01-01', '2015-04-11', '2015-07-20', '2015-10-28', '2016-02-05',
       '2016-05-15', '2016-08-23', '2016-12-01', '2017-03-11', '2017-06-19',
       '2017-09-27', '2018-01-05', '2018-04-15', '2018-07-24', '2018-11-01',
       '2019-02-09', '2019-05-20', '2019-08-28', '2019-12-06', '2020-03-15',
       '2020-06-23', '2020-10-01', '2021-01-09', '2021-04-19', '2021-07-28',
       '2021-11-05', '2022-02-13', '2022-05-24', '2022-09-01', '2022-12-10',
       '2023-03-20', '2023-06-28', '2023-10-06', '2024-01-14', '2024-04-23',
       '2024-08-01'],
      dtype='object')

In [308]:
URL = f'https://iss.moex.com/iss/history/engines/stock/markets/shares/securities/{'SBER'}.json'
params = {
    'from': '2015-01-01', 
    'till': '2024-08-01',
    'interval': 24,  # Дневные данные        
    'iss.meta': 'off',
    'history.columns': 'TRADEDATE,CLOSE'
}


In [688]:
lst = {}
for SECID in top_id:
    tmp = []
    URL = f'https://iss.moex.com/iss/history/engines/stock/markets/shares/securities/{SECID}.json'
    for i in range(len(date)-1):
        params = {
            'from': date[i], 
            'till': date[i+1],
            'interval': 24,  # Дневные данные        
            'iss.meta': 'off',
            'history.columns': 'TRADEDATE,CLOSE'
        }
        try:
            response = requests.get(URL, params)
            dat = response.json()['history']['data']
            tmp += dat
            time.sleep(5)
        except:
            time.sleep(10)
            response = requests.get(URL, params)
            dat = response.json()['history']['data']
            tmp += dat
    lst[SECID] = tmp 


In [689]:
dates = sorted(set([i[0] for i in lst['SBER']]))

In [750]:
data_top = pd.DataFrame({'date':dates})

count = 0
for k in lst:
    count +=1
    tmp = pd.DataFrame(
        {  
        'date':[i[0] for i in lst[k]], 
        k:[i[1] for i in lst[k]]
        })
    tmp = tmp.groupby(['date'],as_index=False).agg({k:'max'})
    tmp.drop_duplicates(subset=['date', k])
    data_top = data_top.merge(tmp, how = 'outer', on = 'date')
    #print(tmp)
    

data_top

,date,SBER,ROSN,LKOH,SIBN,GAZP,NVTK,GMKN,PLZL,YDEX,...,FEES,AFKS,FESH,POSI,UGLD,GCHE,RASP,TRMK,NMTP,GAZS
0,2015-01-05,56.370,197.550,"2,295.000",142.000,133.950,455.500,"8,590.000","1,000.500",NaN,...,0.047,12.250,2.800,NaN,NaN,702.000,23.150,37.500,1.225,NaN
1,2015-01-06,58.280,200.300,"2,345.000",144.600,138.920,476.700,"9,101.000","1,047.000",NaN,...,0.049,12.370,2.800,NaN,NaN,703.000,23.430,37.900,1.205,NaN
2,2015-01-08,65.700,214.950,"2,572.000",155.200,146.460,485.900,"9,550.000","1,060.000",NaN,...,0.050,12.600,2.900,NaN,NaN,686.000,24.250,38.000,1.280,NaN
3,2015-01-09,63.380,207.650,"2,461.000",146.600,141.700,475.100,"9,740.000","1,050.000",NaN,...,0.049,12.800,2.910,NaN,NaN,680.000,23.800,39.200,1.330,NaN
4,2015-01-12,63.040,204.000,"2,477.000",145.500,140.220,470.700,"9,839.000","1,020.000",NaN,...,0.050,12.130,2.730,NaN,NaN,691.000,23.220,38.400,1.275,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414,2024-07-26,NaN,521.400,"6,890.000",684.400,NaN,"1,067.200",NaN,"12,605.500","3,962.500",...,NaN,NaN,NaN,"2,980.400",NaN,"4,556.000",NaN,NaN,NaN,NaN
2415,2024-07-29,NaN,503.200,"6,727.500",671.400,NaN,"1,020.800",NaN,"12,283.000","3,867.000",...,NaN,NaN,NaN,"2,862.200",NaN,"4,400.000",NaN,NaN,NaN,NaN
2416,2024-07-30,NaN,513.950,"6,805.500",676.800,NaN,"1,048.600",NaN,"12,725.000","3,910.000",...,NaN,NaN,NaN,"2,910.000",NaN,"4,395.000",NaN,NaN,NaN,NaN
2417,2024-07-31,NaN,515.950,"6,759.500",684.850,NaN,"1,043.600",NaN,"12,929.500","3,892.000",...,NaN,NaN,NaN,"2,954.000",NaN,"4,400.500",NaN,NaN,NaN,NaN


In [752]:
data_top_50 = pd.DataFrame({'secid':[], 'date':[], 'price':[]})
for k in lst:
    tmp = pd.DataFrame(
        {'secid':k, 
        'date':[i[0] for i in lst[k]], 
        'Cost':[i[1] for i in lst[k]]
        })
    data_top_50 = pd.concat([data_top_50, tmp], axis = 0)


In [754]:
data_top_50

,secid,date,price,Cost
0,SBER,2015-01-05,NaN,NaN
1,SBER,2015-01-05,NaN,56.260
2,SBER,2015-01-05,NaN,56.370
3,SBER,2015-01-06,NaN,NaN
4,SBER,2015-01-06,NaN,57.830
...,...,...,...,...
2438,GAZS,2024-07-26,NaN,NaN
2439,GAZS,2024-07-29,NaN,NaN
2440,GAZS,2024-07-30,NaN,NaN
2441,GAZS,2024-07-31,NaN,NaN


In [424]:
data_top50 = data_top_50.groupby(['secid','date'],as_index=False).agg({'Cost':'max'})

In [758]:
data_top50

,secid,date,Cost
0,AFKS,2015-01-05,12.250
1,AFKS,2015-01-06,12.370
2,AFKS,2015-01-08,12.600
3,AFKS,2015-01-09,12.800
4,AFKS,2015-01-12,12.130
...,...,...,...
85355,YDEX,2024-07-26,"3,962.500"
85356,YDEX,2024-07-29,"3,867.000"
85357,YDEX,2024-07-30,"3,910.000"
85358,YDEX,2024-07-31,"3,892.000"


# Индекс мосбиржи 

In [760]:
URL = f"https://iss.moex.com/iss/history/engines/stock/markets/index/securities/IMOEX.json"
index_mos = []
for i in range(len(date)-1):
    params = {
        'from': date[i], 
        'till': date[i+1],
        'interval': 24,  # Дневные данные        
        'iss.meta': 'off',
        'history.columns': 'TRADEDATE,CLOSE'
    }
    try:
        response = requests.get(URL, params)
        dat = response.json()['history']['data']
        index_mos += dat
        time.sleep(5)
    except:
        time.sleep(10)
        response = requests.get(URL, params)
        dat = response.json()['history']['data']
        index_mos += dat

In [761]:
DF_index = pd.DataFrame({
    'date': [i[0] for i in index_mos],
    'IMOEX': [i[1] for i in index_mos]
    
})

In [762]:
DF_index


,date,IMOEX
0,2015-01-05,"1,435.660"
1,2015-01-06,"1,480.730"
2,2015-01-08,"1,547.390"
3,2015-01-09,"1,515.370"
4,2015-01-12,"1,513.220"
...,...,...
2421,2024-07-26,"2,993.230"
2422,2024-07-29,"2,906.950"
2423,2024-07-30,"2,944.780"
2424,2024-07-31,"2,942.680"


# Сортировка по корреляции 

In [432]:
korrelation = data_top50.merge(DF_index, how = 'inner', on = 'date')

In [434]:
korrelation

,secid,date,Cost,IMOEX
0,AFKS,2015-01-05,12.250,"1,435.660"
1,AFKS,2015-01-06,12.370,"1,480.730"
2,AFKS,2015-01-08,12.600,"1,547.390"
3,AFKS,2015-01-09,12.800,"1,515.370"
4,AFKS,2015-01-12,12.130,"1,513.220"
...,...,...,...,...
85628,YDEX,2024-07-26,"3,962.500","2,993.230"
85629,YDEX,2024-07-29,"3,867.000","2,906.950"
85630,YDEX,2024-07-30,"3,910.000","2,944.780"
85631,YDEX,2024-07-31,"3,892.000","2,942.680"


In [409]:
korrelation.to_csv('korrelation.csv')

In [436]:
tmp = set(korrelation['secid'])

In [440]:
korr = []
for i in tmp:
    d = korrelation.loc[korrelation.secid == i, :].dropna()
    x = d['Cost']
    y = d['IMOEX']
    korr.append([i, np.corrcoef(x, y)[0,1]])
    

In [444]:
sorted_list = sorted(korr, key=lambda x: x[1])

## Посчитать корреляцию для некоторых компаний с лаггом: (индекс мосбиржи месяц назад <-> ценой акции сейчас)

In [349]:
sber = (korrelation[korrelation['secid'] == 'SBER'].reset_index().drop('index',axis = 1))

In [351]:
dano = sber.loc[sber[sber['date'] == '2015-02-02'].index[0] :,].shape[0]
index = sber[sber['date'] == '2015-02-02'].index[0]

In [353]:
sber.loc[index:,'price'].corr(sber.loc[:dano,'price_2'])

0.9217246919732227

In [365]:
gazt =  (korrelation[korrelation['secid'] == 'GAZT'].reset_index().drop('index',axis = 1))
dano = gazt.loc[gazt[gazt['date'] == '2015-02-02'].index[0] :,].shape[0]
index = gazt[gazt['date'] == '2015-02-02'].index[0]
gazt.loc[index:,'price'].corr(gazt.loc[:dano,'price_2'])

0.5893630806866632